In [1]:
import json
import gensim
import os
import json
import tqdm
import time
import numpy as np
import nltk
import random

%matplotlib  inline
import matplotlib.pyplot as plt

from rouge import Rouge 
rouge = Rouge()

In [2]:
% time
with open('./data_pointer_example.txt', 'r', encoding='UTF-8') as f:
    stories = json.load(f)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.15 µs


In [3]:
model_embedding = np.load('Word2vec_pointer.npz')['E']

In [4]:
ts = time.strftime('%Y-%b-%d-%H-%M-%S', time.gmtime())

save_model_path = os.path.join('won', ts)
os.makedirs('./'+save_model_path)

In [5]:
class Vocab(object):

  def __init__(self, vocab_file, max_size):
    self._word_to_id = {}
    self._id_to_word = {}
    self._count = 0 # keeps track of total number of words in the Vocab

    # [UNK], [PAD], [START] and [STOP] get the ids 0,1,2,3.
    for w in [UNKNOWN_TOKEN, PAD_TOKEN, START_DECODING, STOP_DECODING]:
      self._word_to_id[w] = self._count
      self._id_to_word[self._count] = w
      self._count += 1

    # Read the vocab file and add words up to max_size
    with open(vocab_file, 'r', encoding='utf-8') as vocab_f:
      for line in vocab_f:
        pieces = line.split()
        if len(pieces) != 2:
          print ('Warning: incorrectly formatted line in vocabulary file: %s\n' % line)
          continue
        w = pieces[0]
        if w in [SENTENCE_START, SENTENCE_END, UNKNOWN_TOKEN, PAD_TOKEN, START_DECODING, STOP_DECODING]:
          raise Exception('<s>, </s>, [UNK], [PAD], [START] and [STOP] shouldn\'t be in the vocab file, but %s is' % w)
        if w in self._word_to_id:
          raise Exception('Duplicated word in vocabulary file: %s' % w)
        self._word_to_id[w] = self._count
        self._id_to_word[self._count] = w
        self._count += 1
        if max_size != 0 and self._count >= max_size:
          print ("max_size of vocab was specified as %i; we now have %i words. Stopping reading." % (max_size, self._count))
          break

    print ("Finished constructing vocabulary of %i total words. Last word added: %s" % (self._count, self._id_to_word[self._count-1]))

  def word2id(self, word):
    if word not in self._word_to_id:
      return self._word_to_id[UNKNOWN_TOKEN]
    return self._word_to_id[word]

  def id2word(self, word_id):
    if word_id not in self._id_to_word:
      raise ValueError('Id not found in vocab: %d' % word_id)
    return self._id_to_word[word_id]

  def size(self):
    return self._count

  def write_metadata(self, fpath):
    print ("Writing word embedding metadata file to %s..." % (fpath))
    with open(fpath, "w") as f:
      fieldnames = ['word']
      writer = csv.DictWriter(f, delimiter="\t", fieldnames=fieldnames)
      for i in xrange(self.size()):
        writer.writerow({"word": self._id_to_word[i]})

In [6]:
# <s> and </s> are used in the data files to segment the abstracts into sentences. They don't receive vocab ids.
SENTENCE_START = '<s>'
SENTENCE_END = '</s>'

PAD_TOKEN = '[PAD]' # This has a vocab id, which is used to pad the encoder input, decoder input and target sequence
UNKNOWN_TOKEN = '[UNK]' # This has a vocab id, which is used to represent out-of-vocabulary words
START_DECODING = '[START]' # This has a vocab id, which is used at the start of every decoder input sequence
STOP_DECODING = '[STOP]' # This has a vocab id, which is used at the end of untruncated target sequences

# Note: none of <s>, </s>, [PAD], [UNK], [START], [STOP] should appear in the vocab file.

In [7]:
vocab = Vocab('./vocab', 50000)









max_size of vocab was specified as 50000; we now have 50000 words. Stopping reading.
Finished constructing vocabulary of 50000 total words. Last word added: long-delayed


In [8]:
class Example(object):

  def __init__(self, article, abstract_sentences, vocab):
    # Get ids of special tokens
    start_decoding = vocab.word2id(START_DECODING)
    stop_decoding = vocab.word2id(STOP_DECODING)

    # Process the article
    article_words = article.split()
    if len(article_words) > 100:
      article_words = article_words[:100]
    self.enc_len = len(article_words) # store the length after truncation but before padding
    self.enc_input = [vocab.word2id(w) for w in article_words] # list of word ids; OOVs are represented by the id for UNK token

    # Process the abstract
    abstract = ' '.join(abstract_sentences) # string
    abstract_words = abstract.split() # list of strings
    abs_ids = [vocab.word2id(w) for w in abstract_words] # list of word ids; OOVs are represented by the id for UNK token

    # Get the decoder input sequence and target sequence
    self.dec_input, self.target = self.get_dec_inp_targ_seqs(abs_ids, 20, start_decoding, stop_decoding)
    self.dec_len = len(self.dec_input)

    # If using pointer-generator mode, we need to store some extra info
    if True:
      # Store a version of the enc_input where in-article OOVs are represented by their temporary OOV id; also store the in-article OOVs words themselves
      self.enc_input_extend_vocab, self.article_oovs = article2ids(article_words, vocab)

      # Get a verison of the reference summary where in-article OOVs are represented by their temporary article OOV id
      abs_ids_extend_vocab = abstract2ids(abstract_words, vocab, self.article_oovs)

      # Overwrite decoder target sequence so it uses the temp article OOV ids
      _, self.target = self.get_dec_inp_targ_seqs(abs_ids_extend_vocab, 20, start_decoding, stop_decoding)

    # Store the original strings
    self.original_article = article
    self.original_abstract = abstract
    self.original_abstract_sents = abstract_sentences


  def get_dec_inp_targ_seqs(self, sequence, max_len, start_id, stop_id):
    inp = [start_id] + sequence[:]
    target = sequence[:]
    if len(inp) > max_len: # truncate
      inp = inp[:max_len]
      target = target[:max_len] # no end_token
    else: # no truncation
      target.append(stop_id) # end token
    assert len(inp) == len(target)
    return inp, target


  def pad_decoder_inp_targ(self, max_len, pad_id):
    while len(self.dec_input) < max_len:
      self.dec_input.append(pad_id)
    while len(self.target) < max_len:
      self.target.append(pad_id)


  def pad_encoder_input(self, max_len, pad_id):
    while len(self.enc_input) < max_len:
      self.enc_input.append(pad_id)
    if True:
      while len(self.enc_input_extend_vocab) < max_len:
        self.enc_input_extend_vocab.append(pad_id)
        
def article2ids(article_words, vocab):
  ids = []
  oovs = []
  unk_id = vocab.word2id(UNKNOWN_TOKEN)
  for w in article_words:
    i = vocab.word2id(w)
    if i == unk_id: # If w is OOV
      if w not in oovs: # Add to list of OOVs
        oovs.append(w)
      oov_num = oovs.index(w) # This is 0 for the first article OOV, 1 for the second article OOV...
      ids.append(vocab.size() + oov_num) # This is e.g. 50000 for the first article OOV, 50001 for the second...
    else:
      ids.append(i)
  return ids, oovs


def abstract2ids(abstract_words, vocab, article_oovs):
  ids = []
  unk_id = vocab.word2id(UNKNOWN_TOKEN)
  for w in abstract_words:
    i = vocab.word2id(w)
    if i == unk_id: # If w is an OOV word
        #print(abstract_words)
        ids.append(i) 
#       if w in article_oovs: # If w is an in-article OOV
#         vocab_idx = vocab.size() + article_oovs.index(w) # Map to its temporary article OOV number
#         ids.append(vocab_idx)
#       else: # If w is an out-of-article OOV
#         ids.append(unk_id) # Map to the UNK token id
    else:
        ids.append(i)
  return ids

In [9]:
start_decoding = vocab.word2id(START_DECODING)
stop_decoding = vocab.word2id(STOP_DECODING)

input_index=[]
dec_inputs_index=[]
target_index=[]
inputs_len=[]
dec_inputs_len=[]

index_data = {}

zero_len = 0

dec_max_len = 20
enc_max_len = 100

pad_id = vocab.word2id(PAD_TOKEN)
files_group = list(stories.keys())

for file_group in tqdm.tqdm(files_group):
    for file in tqdm.tqdm(list(stories[file_group].keys())):       
        for data in list(stories[file_group][file].keys()):
            if stories[file_group][file][data]['abstract_sentences'] == []:
                print('*********************************************')
                abstract = stories[file_group][file][data]['abstract']
                abstract_sentences = []
                article = stories[file_group][file][data]['article']
                example=Example(article=article,abstract_sentences=abstract_sentences,vocab=vocab)
                example.pad_decoder_inp_targ(dec_max_len,pad_id)
                example.pad_encoder_input(enc_max_len,pad_id)
            else:
                abstract = stories[file_group][file][data]['abstract']
                abstract_sentences = [stories[file_group][file][data]['abstract_sentences'][0]]
                article = stories[file_group][file][data]['article']
                example=Example(article=article,abstract_sentences=abstract_sentences,vocab=vocab)
                example.pad_decoder_inp_targ(dec_max_len,pad_id)
                example.pad_encoder_input(enc_max_len,pad_id)
            if example.enc_len <= 0 or example.dec_len <= 0:
                print(file_group, file, data)
                zero_len +=1
                pass
            else:
                input_index.append(example.enc_input)
                dec_inputs_index.append(example.dec_input)
                target_index.append(example.target)
                inputs_len.append(example.enc_len)
                dec_inputs_len.append(example.dec_len)
    print('********************************************************************************')
    print(len(input_index))
    print('********************************************************************************')

    index_data[file_group] = (input_index,inputs_len,dec_inputs_index,dec_inputs_len,target_index)
    input_index=[]
    dec_inputs_index=[]
    target_index=[]
    inputs_len=[]
    dec_inputs_len=[]

  0%|          | 1/288 [00:00<00:45,  6.25it/s]

./chunked/train_*.bin ./chunked\train_000.bin 598
./chunked/train_*.bin ./chunked\train_000.bin 611
./chunked/train_*.bin ./chunked\train_000.bin 676
./chunked/train_*.bin ./chunked\train_000.bin 687
./chunked/train_*.bin ./chunked\train_000.bin 986
./chunked/train_*.bin ./chunked\train_001.bin 276
./chunked/train_*.bin ./chunked\train_001.bin 339
./chunked/train_*.bin ./chunked\train_001.bin 561
./chunked/train_*.bin ./chunked\train_001.bin 640
./chunked/train_*.bin ./chunked\train_001.bin 825
./chunked/train_*.bin ./chunked\train_001.bin 903



  1%|          | 3/288 [00:00<00:46,  6.14it/s]

./chunked/train_*.bin ./chunked\train_002.bin 44
./chunked/train_*.bin ./chunked\train_002.bin 198
./chunked/train_*.bin ./chunked\train_002.bin 727
./chunked/train_*.bin ./chunked\train_002.bin 941
./chunked/train_*.bin ./chunked\train_002.bin 956
./chunked/train_*.bin ./chunked\train_003.bin 56
./chunked/train_*.bin ./chunked\train_003.bin 152



  2%|▏         | 5/288 [00:00<00:46,  6.09it/s]

...

********************************************************************************
287112
********************************************************************************



  8%|▊         | 1/12 [00:00<00:01,  6.27it/s]

********************************************************************************
13368
********************************************************************************



100%|██████████| 3/3 [00:58<00:00, 19.56s/it]s]

********************************************************************************
11490
********************************************************************************


In [14]:
for file_group in files_group:
    print(file_group)
    (input_index,inputs_len,dec_inputs_index,dec_inputs_len,target_index) = index_data[file_group]
    print(len(input_index))
    start = int(len(input_index)*0.7)
    end = -1
    index_data[file_group] = (input_index[start:end],inputs_len[start:end],dec_inputs_index[start:end],dec_inputs_len[start:end],target_index[start:end])
    (input_index,inputs_len,dec_inputs_index,dec_inputs_len,target_index) = index_data[file_group]
    print(len(input_index))

./chunked/train_*.bin
287112
86133
./chunked/val_*.bin
13368
4010
./chunked/test_*.bin
11490
3447


In [15]:
import torch
from torch.autograd import Variable
import torch.nn.utils.rnn as rnn_utils
import torch.nn as nn
import torch.nn.functional as F

In [16]:
embedding_size = 128
hidden_size = 256
bidirectional = True
batch_size_fit = 1024-256

learning_rate = 0.001

vocab_size = len(vocab._word_to_id)

# <s> and </s> are used in the data files to segment the abstracts into sentences. They don't receive vocab ids.
SENTENCE_START = '<s>'
SENTENCE_END = '</s>'

PAD_TOKEN = '[PAD]' # This has a vocab id, which is used to pad the encoder input, decoder input and target sequence
UNKNOWN_TOKEN = '[UNK]' # This has a vocab id, which is used to represent out-of-vocabulary words
START_DECODING = '[START]' # This has a vocab id, which is used at the start of every decoder input sequence
STOP_DECODING = '[STOP]' # This has a vocab id, which is used at the end of untruncated target sequences

# Note: none of <s>, </s>, [PAD], [UNK], [START], [STOP] should appear in the vocab file.

sos_idx = vocab.word2id(START_DECODING)
eos_idx = vocab.word2id(STOP_DECODING)
pad_idx = vocab.word2id(PAD_TOKEN)
unk_idx = vocab.word2id(UNKNOWN_TOKEN)

In [17]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size,numpy_embedding):
        super(Encoder, self).__init__()
        
        self.model_embedding = torch.from_numpy(numpy_embedding).float()
        
        self.embed = nn.Embedding(vocab_size, embed_size)
        #self.embed.weight = nn.Parameter(self.model_embedding)

        self.lstm = nn.LSTM(input_size=embed_size,  #num_layers = 2,
            hidden_size=hidden_size, batch_first=True, 
            bidirectional=True,)

    def forward(self, x):
        #print(x)
        # input: [b x seq]
        embedded = self.embed(x)
        #print(embedded)
        out, h = self.lstm(embedded) # out: [b x seq x hid*2] (biRNN)
        return out, h


In [18]:
class Hypothesis(object):
    def __init__(self, token_id, hidden_state, cell_state, log_prob):
        self._h = hidden_state
        self._c = cell_state
        self.log_prob = log_prob
        self.full_prediction = token_id # list
        self.survivability = self.log_prob/ float(len(self.full_prediction))

    def extend(self, token_id, hidden_state, cell_state, log_prob):
        return Hypothesis(token_id= self.full_prediction + [token_id],
                          hidden_state=hidden_state,
                          cell_state=cell_state,
                          log_prob= self.log_prob + log_prob)

In [19]:
class Pointer_Decoder(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size,numpy_embedding,lamda = 1):
        super(Pointer_Decoder, self).__init__()
        self.lamda = lamda
        self.model_embedding = torch.from_numpy(numpy_embedding).float()
        
        self.embed = nn.Embedding(vocab_size, embed_size)
        #self.embed.weight = nn.Parameter(self.model_embedding)
        
        self.lstm = nn.LSTM(input_size=embed_size,
            hidden_size=hidden_size*2, batch_first=True)
        
        self.V2V = nn.Linear(hidden_size * 4, hidden_size * 4)
        self.V = nn.Linear(hidden_size * 4, vocab_size)
        
        self.Wh = nn.Linear(2 * hidden_size, 2* hidden_size)
        self.Ws = nn.Linear(hidden_size*2, 2*hidden_size)
        self.w_c = nn.Linear(1, 2*hidden_size)
        self.v = nn.Linear(2*hidden_size, 1)

        self.w_h = nn.Linear(hidden_size*2, 1)
        self.w_s = nn.Linear(hidden_size*2, 1)
        self.w_x = nn.Linear(embedding_size, 1)
        
    def forward(self, x, encoder_outputs, encoder_hidden, encoder_cell_state,encoder_inputs, coverage):
        #print(x)
        batch_size = encoder_outputs.size(0)
        enc_max_len = encoder_outputs.size(1)

        embedded = self.embed(x).view(encoder_outputs.size(0),1, -1)
        self.state = (encoder_hidden.view(-1,batch_size,hidden_size*2), encoder_cell_state.view(-1,batch_size,hidden_size*2))

        enc_proj=self.Wh(encoder_outputs.contiguous().view(batch_size,enc_max_len,-1)) # batch_size x enc_max_len x hidden x 2
        
        decoder_outputs, (decoder_hidden, decoder_cell_state) = self.lstm(embedded, self.state) # out: [b x seq x hid*2] (biRNN)
        
        dec_proj = self.Ws(decoder_outputs).expand_as(enc_proj) # batch_size x enc_max_len x hidden x 2

        cov_proj = self.w_c(coverage.view(-1,1)).view(batch_size,enc_max_len,-1) # batch_size x enc_max_len x hidden x 2

        attn_scores = self.v(F.tanh(enc_proj + dec_proj + cov_proj)).view(batch_size, enc_max_len)# batch_size x enc_max_len
        del enc_proj
        del dec_proj
        del cov_proj
        #enc_mask = encoder_inputs.eq_(pad_idx).clone().type(torch.cuda.ByteTensor) # pad 버림

        #attn_scores = attn_scores.masked_fill_(enc_mask, -float(10000)) # pad 버림
        
        attn_scores = F.softmax(attn_scores)

        context = attn_scores.unsqueeze(1).bmm(encoder_outputs) #batch x 1 x hidden x 2

        p_vocab = F.softmax(self.V(self.V2V(torch.cat((decoder_outputs, context), 2)))) #batch x 1 x vocab
        #p_vocab = F.softmax(self.V(torch.cat((decoder_outputs, context), 2))) #batch x 1 x vocab
        p_gen = F.sigmoid(self.w_h(context).squeeze(2) + self.w_s(decoder_outputs).squeeze(2) + self.w_x(embedded).squeeze(2)) #batch x 1

        weighted_Pvocab = p_vocab.squeeze(1) * p_gen #batch x vocab
        del p_vocab
        weighted_attn  = (1-p_gen)*attn_scores #batch x vocab
        
#         구현예정
#         if self.max_article_oov > 0:
#             ext_vocab = Variable(torch.zeros(batch_size, self.max_article_oov).cuda())				#create OOV (but in-article) zero vectors
#             combined_vocab = torch.cat((weighted_Pvocab, ext_vocab), 1)			
#             del ext_vocab
#         else:
#             combined_vocab = weighted_Pvocab

        combined_vocab  = weighted_Pvocab #batch x vocab
    
        enc_mask = encoder_inputs.clone()
        
        combined_vocab =combined_vocab.scatter_add(1, enc_mask, weighted_attn) #batch x vocab
        del enc_mask
        
        return combined_vocab, decoder_hidden, decoder_cell_state, attn_scores, p_gen, attn_scores
        
        
    def decode_step(self, x, encoder_outputs, encoder_hidden, encoder_cell_state, encoder_inputs):
        #print(x)
        batch_size = encoder_outputs.size(0)
        enc_max_len = encoder_outputs.size(1)
        coverage =  Variable(torch.zeros(batch_size, enc_max_len)).cuda()
        
        embedded = self.embed(x).view(encoder_outputs.size(0),1, -1)
        state = (encoder_hidden.view(-1,batch_size,hidden_size*2), encoder_cell_state.view(-1,batch_size,hidden_size*2))

        enc_proj=self.Wh(encoder_outputs.contiguous().view(batch_size,enc_max_len,-1)) # batch_size x enc_max_len x hidden x 2

        decoder_outputs, (decoder_hidden, decoder_cell_state) = self.lstm(embedded, state) # out: [b x seq x hid*2] (biRNN)

        dec_proj = self.Ws(decoder_outputs).expand_as(enc_proj) # batch_size x enc_max_len x hidden x 2

        cov_proj = self.w_c(coverage.view(-1,1)).view(batch_size,enc_max_len,-1) # batch_size x enc_max_len x hidden x 2

        attn_scores = self.v(F.tanh(enc_proj + dec_proj + cov_proj)).view(batch_size, enc_max_len)# batch_size x enc_max_len
        #print(enc_proj.size(),dec_proj.size(),cov_proj.size())
        enc_mask = encoder_inputs.eq_(pad_idx).detach().type(torch.cuda.ByteTensor) # pad 버림

        attn_scores = attn_scores.masked_fill_(enc_mask, -float(10000)) # pad 버림

        attn_scores = F.softmax(attn_scores)

        context = attn_scores.unsqueeze(1).bmm(encoder_outputs) #batch x 1 x hidden x 2

        p_vocab = F.softmax(self.V(torch.cat((decoder_outputs, context), 2))) #batch x 1 x vocab

        p_gen = F.sigmoid(self.w_h(context).squeeze(2) + self.w_s(decoder_outputs).squeeze(2) + self.w_x(embedded).squeeze(2)) #batch x 1

        weighted_Pvocab = p_vocab.squeeze(1) * p_gen #batch x vocab

        weighted_attn  = (1-p_gen)*attn_scores #batch x vocab

        combined_vocab  = weighted_Pvocab #batch x vocab

        combined_vocab =combined_vocab.scatter_add(1, encoder_inputs, weighted_attn) #batch x vocab


        return combined_vocab, decoder_hidden, decoder_cell_state

    def getOverallTopk(self, vocab_probs, _h, _c, all_hyps, results):
        # return top-k values i.e. top-k over all beams i.e. next step input ids
        # return hidden, cell states corresponding to topk
        probs, inds = vocab_probs.topk(k=2, dim=1)
        probs = probs.log().data
        inds = inds.data
        #inds.add_(1)
        candidates = []

        assert len(all_hyps) == probs.size(0), '# Hypothesis and log-prob size dont match'
        # cycle through all hypothesis in full beam
        for i, hypo in enumerate(probs.tolist()):
            for j, _ in enumerate(hypo):
                new_cand = all_hyps[i].extend(token_id=inds[i,j],
                                              hidden_state=_h[i].unsqueeze(0),
                                              cell_state=_c[i].unsqueeze(0),
                                              log_prob= probs[i,j])
                candidates.append(new_cand)
        # sort in descending order
        candidates = sorted(candidates, key=lambda x:x.survivability, reverse=True)
        new_beam, next_inp = [], []
        next_h, next_c = [], []
        #prune hypotheses and generate new beam
        for h in candidates:
            if h.full_prediction[-1] == eos_idx:
                # weed out small sentences that likely have no meaning
                if len(h.full_prediction)>=5:
                    results.append(h.full_prediction)
            else:
                new_beam.append(h)
                next_inp.append(h.full_prediction[-1])
                next_h.append(h._h.data)
                next_c.append(h._c.data)
            if len(new_beam) >= 2:
                break
        assert len(new_beam) >= 1, 'Non-existent beam'

        return new_beam, torch.LongTensor([next_inp]), results, torch.cat(next_h, 0), torch.cat(next_c, 0)

    def decode(self, encoder_outputs, decoder_hidden, decoder_cell_state, encoder_inputs):
        _input = Variable(torch.LongTensor([[sos_idx]]).cuda(), volatile=True)
        decoded_outputs = []
        all_hyps = [Hypothesis([sos_idx], None, None, 0)]
        for _step in range(dec_max_len):
            print(_input)
            curr_beam_size = _input.size(0)
            enc_states = encoder_outputs.clone()
            beam_enc_states = enc_states.expand(curr_beam_size, enc_states.size(1), enc_states.size(2)).contiguous().detach()
            beam_article_inds  = encoder_inputs.clone().expand(curr_beam_size, x_.size(1)).detach()

            vocab_probs, next_h, next_c = self.decode_step( _input, beam_enc_states, decoder_hidden, decoder_cell_state, beam_article_inds)

            all_hyps, decode_inds, decoded_outputs, init_h, init_c = self.getOverallTopk(vocab_probs, next_h.view(curr_beam_size,1,-1), \
                                                                                    next_c.view(curr_beam_size,1,-1), all_hyps, decoded_outputs)
            #decode_inds.masked_fill_((decode_inds > vocab_size), unk_idx)
            
            decode_inds = decode_inds.t()
            _input = Variable(decode_inds.cuda(), volatile=True)
            decoder_hidden = Variable(init_h,volatile=True)
            decoder_cell_state = Variable(init_c,volatile=True)

        non_terminal_output = [item.full_prediction for item in all_hyps]
        all_outputs = decoded_outputs + non_terminal_output
        return all_outputs

In [20]:
encoder = Encoder(vocab_size, embedding_size, hidden_size, model_embedding).cuda()

In [21]:
decoder = Pointer_Decoder(vocab_size, embedding_size, hidden_size,model_embedding,lamda = 1).cuda()

In [22]:
encoder,decoder

(Encoder(
   (embed): Embedding(50000, 128)
   (lstm): LSTM(128, 256, batch_first=True, bidirectional=True)
 ), Pointer_Decoder(
   (embed): Embedding(50000, 128)
   (lstm): LSTM(128, 512, batch_first=True)
   (V2V): Linear(in_features=1024, out_features=1024)
   (V): Linear(in_features=1024, out_features=50000)
   (Wh): Linear(in_features=512, out_features=512)
   (Ws): Linear(in_features=512, out_features=512)
   (w_c): Linear(in_features=1, out_features=512)
   (v): Linear(in_features=512, out_features=1)
   (w_h): Linear(in_features=512, out_features=1)
   (w_s): Linear(in_features=512, out_features=1)
   (w_x): Linear(in_features=128, out_features=1)
 ))

In [23]:
NLL = torch.nn.NLLLoss(ignore_index = pad_idx)

optimizer_encoder = torch.optim.Adam(encoder.parameters(), lr=learning_rate)
optimizer_decoder = torch.optim.Adam(decoder.parameters(), lr=learning_rate)

In [24]:
import random

def batch(batch_size,input_var,dec_input_var,target_var,length_var, dec_length_var):

    shuffle_list = list(zip(input_var,dec_input_var,target_var,length_var,dec_length_var))
    random.shuffle(shuffle_list)
    
    start = 0
    end = batch_size
    #if len(input_var)%32 != 0:
    while end < len(input_var):
        batch_input = []
        batch_input_dec = []
        batch_target = []
        batch_length = []
        batch_length_dec = []
        
        batch_shuffle = shuffle_list[start:end]
        
        for i,j,k,n,m in batch_shuffle:
            batch_input.append(i)
            batch_input_dec.append(j)
            batch_target.append(k)
            batch_length.append(n)
            batch_length_dec.append(m)
            
        temp = end
        end  = end + batch_size
        start = temp
        yield batch_input, batch_input_dec, batch_target, batch_length, batch_length_dec
        
    if end >= len(input_var):
        batch_input = []
        batch_input_dec = []
        batch_target = []
        batch_length = []
        batch_length_dec = []
        batch_shuffle = shuffle_list[start:]
        
        for i,j,k,n,m in batch_shuffle:
            batch_input.append(i)
            batch_input_dec.append(j)
            batch_target.append(k)
            batch_length.append(n)
            batch_length_dec.append(m)
        yield batch_input, batch_input_dec, batch_target, batch_length, batch_length_dec

In [25]:
epochs = 200
max_target_len = 20
clip = 2.0
teacher_forcing_ratio = 0.0

In [ ]:
paths = ['./chunked/train_*.bin', './chunked/val_*.bin']

In [ ]:
step = 0
var_losses = []
train_losses = []
avg_losses = []
coverege_losses = []
iteration = 0
lamda = 1
for epoch in tqdm.tqdm(range(epochs+1)):
    optimizer_encoder.zero_grad()
    optimizer_decoder.zero_grad()
    for path in paths :
        print(path)
        input_index, inputs_len, dec_inputs_index, dec_inputs_len, target_index = index_data[path]
        
        for batch_x, batch_y_x, batch_y, batch_len, batch_len_y in batch(batch_size_fit, input_index, \
                                                                         dec_inputs_index,target_index, \
                                                                         inputs_len, dec_inputs_len):
            optimizer_encoder.zero_grad()
            optimizer_decoder.zero_grad()

            iteration = iteration + 1

            if path == paths[0]:
                encoder.train()
                decoder.train()
            else:
                encoder.eval()
                decoder.eval()

            x_ = Variable(torch.cuda.LongTensor(batch_x))
            #x_index = Variable(torch.cuda.LongTensor(batch_x))
            y_ = Variable(torch.cuda.LongTensor(batch_y)).transpose(1,0)
            batch_size = x_.size(0)


            encoder_outputs, (encoder_hidden, encoder_cell_state) = encoder(x_)
            
            decoder_input=Variable(torch.cuda.LongTensor(batch_size)).fill_(sos_idx)
            decoder_hidden = encoder_hidden#[-2:]
            decoder_cell_state = encoder_cell_state#[-2:]
            
            coverage =  Variable(torch.zeros(batch_size, enc_max_len)).cuda()
            
            loss = 0
            coverege_loss_ = 0
            use_teacher_forcing = random.random() < teacher_forcing_ratio
            if use_teacher_forcing:# or path == paths[1]:
                print('use_teacher_forcing, '+path)
                for i in range(max_target_len):
                    P_vocab, decoder_hidden, decoder_cell_state, attn_scores, p_gen, attn_scores = decoder(\
                                                        decoder_input,encoder_outputs, decoder_hidden, decoder_cell_state,\
                                                                                                        x_,coverage)

                    
                    
                    _c_loss , index=torch.stack((coverage,attn_scores),2).min(2)
                    #print(index[0])
                    coverege_loss = _c_loss.type(torch.cuda.DoubleTensor).sum(1)
                    coverege_loss = coverege_loss.sum().div(batch_size)
                    coverage =coverage+ attn_scores
                    topv, topi = P_vocab.data.topk(1)

                    nllloss = NLL(torch.log(P_vocab),y_[i])
                    
                    total_loss = nllloss + lamda*coverege_loss.type(torch.cuda.FloatTensor)
                    loss += total_loss
                    
                    coverege_loss_ += coverege_loss
                    #decoder_hidden = decoder_hidden.squeeze(0)
                    decoder_input = Variable(topi).squeeze(0)
            else:
                for i in range(max_target_len):
                    #print(coverage)
                    P_vocab, decoder_hidden, decoder_cell_state, attn_scores, p_gen, attn_scores = decoder(\
                                                                decoder_input,encoder_outputs, decoder_hidden, decoder_cell_state,\
                                                                                                           x_,coverage)
                    #print('attn_scores')
                    #print(attn_scores)
                    
                    _c_loss , index=torch.stack((coverage,attn_scores),2).min(2)
                    #print(index[0].sum())
                    coverege_loss = _c_loss.type(torch.cuda.DoubleTensor).sum(1)
                    coverege_loss = coverege_loss.sum().div(batch_size)
                    coverage =coverage+ attn_scores
                    topv, topi = P_vocab.data.topk(1)

                    nllloss = NLL(torch.log(P_vocab),y_[i])
                    total_loss = nllloss + lamda*coverege_loss.type(torch.cuda.FloatTensor)
                    loss += total_loss
                    
                    coverege_loss_ += coverege_loss
                    #decoder_hidden = decoder_hidden.squeeze(0)
                    decoder_input = y_[i]
            
            if path == paths[0]:

                loss.backward()
                #print('a')
                torch.nn.utils.clip_grad_norm(encoder.parameters(), clip)
                torch.nn.utils.clip_grad_norm(decoder.parameters(), clip)
                optimizer_encoder.step()
                optimizer_decoder.step()
                
                loss = loss.data[0]/max_target_len
                c_loss = coverege_loss_.data[0]/max_target_len
                coverege_losses.append(c_loss)
                train_losses.append(loss)
                

                
                step += 1

                if iteration % 10 == 0 or iteration == (len(input_index)-1)//batch_size:
                    print("Batch %04d/%i, Loss %9.4f, coverage Loss %9.4f"%( iteration, (len(input_index)-1)//batch_size_fit, loss, c_loss))
                    np.savez(L=train_losses,file='./train_loss.npz')
                    np.savez(L=coverege_losses,file='./coverege_loss.npz')
                    checkpoint_path_encoder = os.path.join(save_model_path, "enc_E%i.pytorch"%(epoch))
                    checkpoint_path_decoder = os.path.join(save_model_path, "dec_E%i.pytorch"%(epoch))
                    torch.save(encoder, checkpoint_path_encoder)
                    torch.save(decoder, checkpoint_path_decoder)
                    


                del nllloss
                del loss
                del x_
                del y_
                del total_loss
                
            else:

                loss = loss.data[0]/max_target_len

                var_losses.append(loss)

                step += 1

                if iteration % 10 == 0 or iteration == (len(input_index)-1)//batch_size:
                    print("Valid Batch %04d/%i, Loss %9.4f"%( iteration, (len(input_index)-1)//batch_size_fit, loss))
                    np.savez(L=var_losses,file='./var_loss.npz')
                    


                del nllloss
                del loss
                del x_
                del y_
                del total_loss
                
            optimizer_encoder.zero_grad()
            optimizer_decoder.zero_grad()
        iteration = 0
        
    print("Model saved at %s"%save_model_path)
    print("Epoch %02d/%i, Mean ELBO %9.4f"%( epoch, epochs, np.mean(np.array(var_losses))))
    avg_losses.append(np.mean(np.array(var_losses)))
    np.savez(L=avg_losses,file='./avg_losses.npz')
        

  0%|          | 0/201 [00:00<?, ?it/s]

./chunked/train_*.bin


/opt/conda/envs/pytorch-py3.6/lib/python3.6/site-packages/ipykernel_launcher.py:49: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/opt/conda/envs/pytorch-py3.6/lib/python3.6/site-packages/ipykernel_launcher.py:53: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Batch 0010/112, Loss    5.9769, coverage Loss    0.9496


/opt/conda/envs/pytorch-py3.6/lib/python3.6/site-packages/torch/serialization.py:158: UserWarning: Couldn't retrieve source code for container of type Encoder. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/opt/conda/envs/pytorch-py3.6/lib/python3.6/site-packages/torch/serialization.py:158: UserWarning: Couldn't retrieve source code for container of type Pointer_Decoder. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


...

Model saved at won/2018-Aug-01-01-05-16
Epoch 12/200, Mean ELBO    3.5239
./chunked/train_*.bin
Batch 0010/112, Loss    1.7231, coverage Loss    0.0621
Batch 0020/112, Loss    1.7144, coverage Loss    0.0611
Batch 0030/112, Loss    1.7637, coverage Loss    0.0609
Batch 0040/112, Loss    1.8077, coverage Loss    0.0636
Batch 0050/112, Loss    1.8114, coverage Loss    0.0617
Batch 0060/112, Loss    1.7683, coverage Loss    0.0590
Batch 0070/112, Loss    1.8153, coverage Loss    0.0612
Batch 0080/112, Loss    1.8322, coverage Loss    0.0612
Batch 0090/112, Loss    1.8663, coverage Loss    0.0617
Batch 0100/112, Loss    1.8455, coverage Loss    0.0631
Batch 0110/112, Loss    1.8858, coverage Loss    0.0655
Batch 0112/112, Loss    1.8850, coverage Loss    0.0615
./chunked/val_*.bin
Valid Batch 0005/5, Loss    3.8539


  7%|▋         | 14/201 [53:07<11:49:40, 227.70s/it]

Model saved at won/2018-Aug-01-01-05-16
Epoch 13/200, Mean ELBO    3.5374
./chunked/train_*.bin
Batch 0010/112, Loss    1.7029, coverage Loss    0.0590
Batch 0020/112, Loss    1.7124, coverage Loss    0.0571
Batch 0030/112, Loss    1.6960, coverage Loss    0.0573
Batch 0040/112, Loss    1.7211, coverage Loss    0.0560
Batch 0050/112, Loss    1.6589, coverage Loss    0.0542
Batch 0060/112, Loss    1.6795, coverage Loss    0.0548
Batch 0070/112, Loss    1.7096, coverage Loss    0.0576
Batch 0080/112, Loss    1.7381, coverage Loss    0.0534
Batch 0090/112, Loss    1.7635, coverage Loss    0.0572
Batch 0100/112, Loss    1.7362, coverage Loss    0.0537
Batch 0110/112, Loss    1.7850, coverage Loss    0.0558
Batch 0112/112, Loss    1.7869, coverage Loss    0.0551
./chunked/val_*.bin
Valid Batch 0005/5, Loss    3.8050


  7%|▋         | 15/201 [56:55<11:45:53, 227.70s/it]

Model saved at won/2018-Aug-01-01-05-16
Epoch 14/200, Mean ELBO    3.5519
./chunked/train_*.bin
Batch 0010/112, Loss    1.5877, coverage Loss    0.0513
Batch 0020/112, Loss    1.5674, coverage Loss    0.0512
Batch 0030/112, Loss    1.6097, coverage Loss    0.0522
Batch 0040/112, Loss    1.6238, coverage Loss    0.0478
Batch 0050/112, Loss    1.6845, coverage Loss    0.0523
Batch 0060/112, Loss    1.7038, coverage Loss    0.0540
Batch 0070/112, Loss    1.6523, coverage Loss    0.0522
Batch 0080/112, Loss    1.6892, coverage Loss    0.0513
Batch 0090/112, Loss    1.6367, coverage Loss    0.0516
Batch 0100/112, Loss    1.6660, coverage Loss    0.0542


In [ ]:
save_model_path,epoch

In [ ]:
save_model_path = os.path.join('won', '2018-Jul-31-09-36-35')

In [ ]:
checkpoint_path_encoder = os.path.join(save_model_path, 'enc_E22.pytorch')
checkpoint_path_decoder = os.path.join(save_model_path, 'dec_E22.pytorch')

In [ ]:
encoder = torch.load(checkpoint_path_encoder)
decoder = torch.load(checkpoint_path_decoder)

In [ ]:
encoder.eval()
decoder.eval()
print(encoder,decoder)

# greedy generator

In [ ]:
(input_index,inputs_len,dec_inputs_index,dec_inputs_len,target_index) = index_data[files_group[0]]

In [ ]:
start = 0
end = 1000

In [ ]:
x_ = Variable(torch.cuda.LongTensor(input_index[start:end]))

batch_size = x_.size(0)

encoder_outputs, (encoder_hidden, encoder_cell_state) = encoder(x_)

In [ ]:
decoder_input=Variable(torch.cuda.LongTensor(batch_size)).fill_(sos_idx)
decoder_hidden = encoder_hidden#[-2:]
decoder_cell_state = encoder_cell_state#[-2:]
coverage =  Variable(torch.zeros(batch_size, enc_max_len)).cuda()

a=decoder.decode( encoder_outputs, decoder_hidden, decoder_cell_state, x_)

b=list(map(vocab.id2word,a[0]))
c=list(map(vocab.id2word,a[1]))
d=list(map(vocab.id2word,target_index[start:end][0]))

rouge.get_scores(' '.join(d), ' '.join(c))

In [ ]:

decoded_words = []
decoder_attentions = torch.zeros(encoder_outputs.size(0),dec_max_len+20, enc_max_len)
for i in range(dec_max_len):
    #print(decoder_input,decoder_hidden)
    P_vocab, decoder_hidden, decoder_cell_state, attn_scores, p_gen, decoder_attention = decoder(\
                                                        decoder_input,encoder_outputs, decoder_hidden, decoder_cell_state,\
                                                                                                        x_,coverage)
    coverage = coverage+attn_scores
    decoder_attentions[:attn_scores.size(0),i,:attn_scores.size(1)] += attn_scores.cpu().data
    topv, topi = P_vocab.data.topk(1)
    #ni = topi[0]
    '''
    if ni == eos_idx:
        decoded_words.append(_EOS_)
        break
    else:
        decoded_words.append(index2word[ni])
    '''
    temp = []
    if len(topi.size()) == 1:
        temp.append(vocab.id2word(topi.cpu().numpy()[0]))
    else:
        for top in topi.cpu().numpy():
            temp.append(vocab.id2word(top[0]))
    decoded_words.append(temp)
    decoder_input = Variable(topi).cuda()
    decoder_hidden = decoder_hidden.squeeze()
    if len(decoder_hidden.size()) == 1:
        decoder_hidden = decoder_hidden.unsqueeze(0)
    else: 
        decoder_hidden = decoder_hidden
del coverage


In [ ]:
inferences = []
for decoded_sent in np.array(decoded_words).transpose():
    for i,word in enumerate(decoded_sent):
        if word == STOP_DECODING:
            decoded_sent = decoded_sent[:i+1]

    inferences.append(list(decoded_sent))


targets_result = []
for inputs in target_index[start:end]:
    result = []
    for word in inputs:
        if word == eos_idx:
            break
        else:
            result.append(vocab.id2word(word))
    targets_result.append(result)    

inputs_result = []
for inputs in input_index[start:end]:
    result = []
    for word in inputs:
        result.append(vocab.id2word(word))
    inputs_result.append(result)

In [ ]:
for source,target,inference, decoder_attention in zip(inputs_result[:10], targets_result[:10], inferences[:10], decoder_attentions[:10]):
    print('*********************************************************************')
    print('source : '+' '.join(source))
    
    print('*********************************************************************')
    print('target : '+' '.join(target))
    
    print('*********************************************************************')
    print('inference : '+' '.join(inference))
    
    print('*********************************************************************')
    scores = rouge.get_scores(' '.join(target), ' '.join(inference))
    for score in scores[0]:
        print(score +' : '+str(scores[0][score]))

    plt.matshow(decoder_attention.numpy())

In [ ]:
r1_recall = []
r2_recall = []
rl_recall = []

r1_f1 = []
r2_f1 = []
rl_f1 = []

error = 0

for source,target,inference in tqdm.tqdm(zip(inputs_result, targets_result, inferences)):#, decoder_attentions)):
    try:
        scores = rouge.get_scores(' '.join(target), ' '.join(inference))
    except ValueError:
        error +=1
        print('******************************************')
        print('target'+' '.join(target))
        print('inference'+' '.join(inference))
        pass
    r1_recall.append(scores[0]['rouge-1']['r'])
    r2_recall.append(scores[0]['rouge-2']['r'])
    rl_recall.append(scores[0]['rouge-l']['r'])
    
    r1_f1.append(scores[0]['rouge-1']['f'])
    r2_f1.append(scores[0]['rouge-2']['f'])
    rl_f1.append(scores[0]['rouge-l']['f'])
print('error sentence : ' + str(error))

In [ ]:
print('ROUGE-1 recall : ' + str(sum(r1_recall)*100 / len(r1_recall)))
print('ROUGE-1 F1 : ' + str(sum(r1_f1)*100 / len(r1_f1)))
print('********************************************************************')
print('ROUGE-2 recall : ' + str(sum(r2_recall)*100 / len(r2_recall)))
print('ROUGE-2 F1 : ' + str(sum(r2_f1)*100 / len(r2_f1)))
print('********************************************************************')
print('ROUGE-l recall : ' + str(sum(rl_recall)*100 / len(rl_recall)))
print('ROUGE-l F1 : ' + str(sum(rl_f1)*100 / len(rl_f1)))

# 'won/2018-Jul-30-11-30-40' epoch 9
ROUGE-1 recall : 25.147463257859798
ROUGE-1 F1 : 26.152656594511935
********************************************************************
ROUGE-2 recall : 11.111168986470044
ROUGE-2 F1 : 11.87687849599647
********************************************************************
ROUGE-l recall : 23.374466815236698
ROUGE-l F1 : 23.332530990714563

# 'won/2018-Jul-31-01-09-08' epoch18
ROUGE-1 recall : 27.181338680580858
ROUGE-1 F1 : 23.24354440201661
********************************************************************
ROUGE-2 recall : 10.285292567935505
ROUGE-2 F1 : 10.084292863650939
********************************************************************
ROUGE-l recall : 25.50797278681479
ROUGE-l F1 : 20.400153250444962

# 'won', '2018-Jul-31-06-07-54' epoch 14
ROUGE-1 recall : 24.045028589331288
ROUGE-1 F1 : 26.872889411505124
********************************************************************
ROUGE-2 recall : 10.375095347295948
ROUGE-2 F1 : 11.756716997867436
********************************************************************
ROUGE-l recall : 22.083905535277424
ROUGE-l F1 : 23.46146722706788